# **Methylation Biomarkers for Predicting Cancer**

## **Random Forest for Feature Selection**

**Author:** Meg Hutch

**Date:** February 14, 2020

**Objective:** Use random forest to select genes for features in our deep learning classifier.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, accuracy_score, auc, precision_recall_fscore_support, f1_score, log_loss
from sklearn.metrics import make_scorer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score

In [ ]:
# set working directory for git hub
import os
os.chdir('/home/mrh1996/')
#os.chdir('C:\\Users\\User\\Box Sync/Projects/Multi_Cancer_DL/')
os. getcwd()

**Import the training data**

In [ ]:
mcTrain = pd.read_csv('Multi_Cancer_DL/02_Processed_Data/mcTrain_70_30.csv')

**Drop Un-neccessary columns**

In [ ]:
mcTrain = mcTrain.drop(columns=["dilute_library_concentration", "age", "gender", "frag_mean"])

**Split Data into X inputs and Y outputs (diagnosis classification)**

In [ ]:
mcTrain_x = mcTrain.drop(columns=["diagnosis"])
mcTrain_y = mcTrain[['seq_num','diagnosis']]

**Code the Categorical Data**

In [ ]:
# Replace each outcome target with numerical value
mcTrain_y = mcTrain_y.replace('HEA', 0)
mcTrain_y = mcTrain_y.replace('CRC', 1)
mcTrain_y = mcTrain_y.replace('ESCA', 2)
mcTrain_y = mcTrain_y.replace('HCC', 3)
mcTrain_y = mcTrain_y.replace('STAD', 4)
mcTrain_y = mcTrain_y.replace('GBM', 5)
mcTrain_y = mcTrain_y.replace('BRCA', 6)

**Convert seq_num id to index**

In [ ]:
mcTrain_x = mcTrain_x.set_index('seq_num')
mcTrain_y = mcTrain_y.set_index('seq_num')

**Split Training Data into a training/validation**

In [ ]:
from sklearn.model_selection import train_test_split
np.random.seed(21420)
X_train, X_test, y_train, y_test = train_test_split(mcTrain_x, mcTrain_y, test_size=0.25, random_state=25, shuffle = True, stratify = mcTrain_y)

**Examine Disease Distributions After Training/Testing Split**

In [ ]:
y_train_perc = y_train.groupby(['diagnosis']).size()/len(y_train)*100
y_test_perc = y_test.groupby(['diagnosis']).size()/len(y_test)*100

#print(y_train_perc)
#print(y_test_perc)

**One-hot encode y classes**

In [ ]:
from sklearn import preprocessing
y_train_multi = preprocessing.label_binarize(y_train, classes=[0, 1, 2, 3, 4, 5, 6])
y_test_multi = preprocessing.label_binarize(y_test, classes=[0, 1, 2, 3, 4, 5, 6])

**Convert to arrays**

In [ ]:
# save copy of X_train - this will be used for feature selection down the line
X_train_orig = X_train

# Convert all to arrays
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values 
y_test = y_test.values

# Convert y_train to 1D
y_train = y_train.ravel()

# test
#print(y_test.reshape(1,-1)) 

# Feb 22 test
#y_train = y_train.reshape(1,-1)
#y_test = y_test.reshape(1,-1)

In [ ]:
#X_train.shape
#y_train.shape
y_test.shape

# **Random Forest**

The hyperparameter tuning function was adapted from Garrett's modeling lecture:

https://github.com/geickelb/HSIP442_guest_lecture/blob/master/notebooks/modeling.ipynb

scoring parameter for multi-classification: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter (will try f1_samples and precision_samples and/or just accuracy)

**Define Random Forest Hypertuning Function**

In [ ]:
def hypertuning_fxn(X, y, nfolds, model , param_grid, scoring = 'accuracy', verbose=False): 
    """function that uses GridSearchCV to test a specified param_grid of hyperparameters and choose the optimal one based on nfolds cross-validation results. 

    Keyword arguments:
    model -- a 'fitted' sklearn model object 
    X -- predictor matrix (dtype='numpy array', required)
    y -- outcome vector (dtype='numpy array', required)
    cv -- if True, prints a the roc_auc score from 10-fold crossvalidation (dtype='boolean', default='True')
    """
    
    from sklearn.model_selection import KFold, GridSearchCV
    np.random.seed(12345)

    grid_search = GridSearchCV(estimator= model,
                                     param_grid=param_grid,
                                     cv=KFold(nfolds),
                                     scoring=scoring,
                                     return_train_score=True,
                                     n_jobs = -1)

    #OneVsRestClassifier(grid_search.fit(X, y))   
    grid_search.fit(X, y)
    print(" scorer function: {}".format(scoring))
    print(" ##### CV performance: mean & sd scores #####")

    means = grid_search.cv_results_['mean_test_score']
    stds = grid_search.cv_results_['std_test_score']
    print('best cv score: {:0.3f}'.format(grid_search.best_score_))
    print('best cv params: ', grid_search.best_params_)

    worst_index=np.argmin(grid_search.cv_results_['mean_test_score'])
    print('worst cv score: {:0.3f}'.format(grid_search.cv_results_['mean_test_score'][worst_index]))
    print('worst cv params: ', grid_search.cv_results_['params'][worst_index])
    ##
    if verbose==True:
        for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

    return(grid_search)

**Tune Hyperparameters**

In [ ]:
### tuning RF hyperparameters
# Number of trees in random forest
n_estimators = [10] #  100, 300, 500, 1000
# Number of features to consider at every split
max_features = [3, 10, 'auto'] # 'auto' which is equivalent to sqrt(n_features)
# Maximum number of levels in tree
max_depth = [5, 8, 15, 25, 30]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 5, 10, 15]

param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

model = RandomForestClassifier(criterion='entropy', random_state=12345)

rf_hyper=hypertuning_fxn(X_train, y_train, nfolds=10, model=model , param_grid=param_grid, scoring='accuracy')

**Return the Best Estimator**

In [ ]:
rf_hyper.best_estimator_

# **Evaluate Model**

In [ ]:
def evaluate_model(model, x, y, cv=True):
    """prints common binary classification evaluation metrics and an ROC curve. 

    Keyword arguments:
    model -- a 'fitted' sklearn model object 
    x -- predictor matrix (dtype='numpy array', required)
    y -- outcome vector (dtype='numpy array', required)
    cv -- if True, prints a the roc_auc score from 10-fold crossvalidation (dtype='boolean', default='True')
    """
    import sklearn.metrics
    from sklearn.metrics import log_loss, average_precision_score, precision_recall_curve
    from sklearn.model_selection import cross_val_score

    if cv==True:
        cv_results= cross_val_score(model, x, y, scoring='roc_auc', cv=10)
        print("across 10 fold cv on trainingset, the model had \n", 
             "mean auroc: {:0.3f}".format(np.mean(cv_results)), "\n",
             "std auroc: {:0.3f}".format(np.std(cv_results))
             )

        base_cv_score=np.mean(cross_val_score(model, x, y, scoring='roc_auc', cv=10)) 

    
    print("###metrics on provided dataset:###")
    ##basic model performance
    y_hat = model.predict(x) # predicted classes using default 0.5 threshold
    y_proba = model.predict_proba(x)[:, 1] #predicted probabilities
    errors = abs(y_hat - y)
    mape = 100 * np.mean(errors / y) # mean absolute percentage error
    accuracy = 100 - mape 
    auc=roc_auc_score(y, y_proba, multi_class = 'ovr', average = 'macro')
    loss= log_loss(y, y_hat)

    print ('the AUC is: {:0.3f}'.format(auc))
    print ('the logloss is: {:0.3f}'.format(loss))
    print("confusion matrix:\n ", confusion_matrix(y, y_hat))
    print("classification report:\n ", classification_report(y,y_hat, digits=3))

    ez_roc(model, x, y, pos_label=1) #plotting roc curve
    plt.show()
    ez_prc(model, x, y, pos_label=1) #plotting roc curve
    plt.show()

In [ ]:
evaluate_model(rf_hyper.best_estimator_,X_test,y_test.ravel(), cv=False)

In [ ]:
X_test.shape
#y_test.ravel().shape
#X_train.shape
#y_train.shape

In [ ]:
y_test.shape

In [ ]:
#https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
#https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html#sklearn.multiclass.OneVsRestClassifier   

from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(rf_hyper.best_estimator_.fit(X_train, y_train))
#classifier = OneVsRestClassifier(rf.fit(X_train, y_train))
#yprob = clf.predict(X_test)
#clf.fit.predict_proba(X_test)

#clf = OneVsRestClassifier(rf()).fit(X, y)

In [ ]:
clf

In [ ]:
macro_roc_auc_ovr = roc_auc_score(y_test, y_prob, multi_class="ovr",
                                  average="macro")

In [ ]:
#rf = multi_target_forest.fit(X_train, y_train)
#rf = rf_hyper.best_estimator_fit(X_train, y_train)
predictions = rf_hyper.best_estimator_.predict(X_test)
print(predictions.shape)
print(predictions)

In [ ]:
errors = abs(predictions - y_test)
#Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

In [ ]:
accuracy_score(y_test, predictions)
#roc_auc_score(y_test, predictions, multi_class = 'ovr') # multi_class must be in ('ovo', 'ovr')
clf = OneVsRestClassifier(rf_hyper.best_estimator_.fit(X_train, y_train))
clf = clf.fit(X_train, y_train)
clf_preds = clf.predict(X_test)
#https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html#sklearn.multiclass.OneVsRestClassifier

In [ ]:
print(rf_hyper.classes_)
rf_probs = rf_hyper.best_estimator_.predict_proba(X_test)[:,]
rf_probs

**Note:** Computes the AUC of each class against the rest [3] [4]. This treats the multiclass case in the same way as the multilabel case. Sensitive to class imbalance even when average == 'macro', because class imbalance affects the composition of each of the ‘rest’ groupings"

In [ ]:
roc_auc_score(y_test, rf_probs, multi_class = 'ovr', average = 'macro') # multi_class must be in ('ovo', 'ovr')

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, rf_probs)
roc_auc = auc(fpr, tpr)
    
plt.title('ROC curve')
ax1= plt.plot(fpr, tpr, 'b', label = '%s AUC = %0.3f' % (model_name, roc_auc), linewidth=2)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
return()

In [ ]:
evaluate_model(rf_hyper.best_estimator_,X_test,y_test, cv=False)

In [ ]:
# Probabilities for each class - this shows how predictions were made - not sure we need to do much more here
rf_probs = rf_hyper.best_estimator_.predict_proba(X_test)[:, 1]
# convert to an array
rf_probs = np.asarray(rf_probs)
# this helps reduce the list
#rf_probs = np.amax(rf_probs, axis=0)
# convert to a dataframe
#rf_probs = pd.DataFrame(rf_probs)
#print(rf_probs)

# Calculate the absolute errors
#errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
#print('Mean Absolute Error:', round(np.mean(errors), 2))

In [ ]:
rf_probs

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
#f1_score(y_test, predictions, average = 'samples') 

#accuracy_score(y_test, predictions)
#roc_auc_score(y_test, predictions) # multi_class must be in ('ovo', 'ovr')
clf = OneVsRestClassifier(rf.fit(X_train, y_train))
clf = clf.fit(X_train, y_train)
clf_preds = clf.predict_proba(X_test)[: 1]
#https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html#sklearn.multiclass.OneVsRestClassifier

In [ ]:
clf_preds2 = clf.predict_proba(X_test)[:, 1]

In [ ]:
clf_preds2.shape

In [ ]:
rf_probsx = np.array(rf_probs)
#rf_probsx = rf_probsx.ravel()
rf_probsx.shape
#predictions = np.array(predictions)

In [ ]:
# Convert y_test to 1D
#y_test = y_test.ravel()
#y_test.shape

In [ ]:
clf_preds = clf_preds.ravel()

In [ ]:
confusion_matrix(y_test, predictions)

In [ ]:
clf_preds

In [ ]:
y_test = y_test.ravel()
y_test.shape

In [ ]:
rf_probs.shape

In [ ]:
y_test

# **Examine Important Features**

Feature Importance for Multi-class classification:
https://stackoverflow.com/questions/54562464/can-i-show-feature-importance-for-multioutputclassifier

MultiOutputClassifier objects have an attribute called estimators_. If you run multi_forest.estimators_, you will get a list containing an object for each of your RandomForest classifiers.

For each of these RandomForest classifier objects, you can access its feature importances through the feature_importances_ attribute.

In [ ]:
# create empty list
feat_impts = [] 
# bind all rf estimators for each classifier (each multi-class output - in our case 7)
for clf in rf.estimators_:
    feat_impts.append(clf.feature_importances_)

# calculate the mean of features across genes
feat = np.mean(feat_impts, axis=0)
# create a list of features (gene names)
features = list(X_train_orig.columns.values) 
# add gene names to the means
feat_importances = pd.Series(feat, index=X_train_orig.columns)  

# plot feature importance for nlargest means 
feat_importances.nlargest(25).plot(kind='barh')